In [1]:
# import Phase-1 utilities
LIGHT_IMPORT = True        # 
import pandas as pd
cands = pd.DataFrame(); uid = 1234   # placeholders for stray debug prints

%run -i "Calc_Embeddings_Model Training.ipynb"

print("df shape:", df.shape if 'df' in globals() else "df missing")
needed = ["top_k_semantic_neighbors_by_text","filter_by_orientation","filter_by_location",
    "filter_by_age","compute_component_scores","apply_weights","get_matches"]
print({n: (n in globals()) for n in needed})

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [2]:
#OkCupid scorer
LIGHT_IMPORT = True
%run -i "ok_cupid_algo.ipynb"
print("okcupid_score available?", "okcupid_score" in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [3]:
#Hinge (supervised) scorer
LIGHT_IMPORT = True
%run -i "hinge_algo.ipynb"

if "hinge_prob" in globals() and "hinge_score" not in globals():
    hinge_score = hinge_prob

print("hinge_score available?", "hinge_score" in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

,coef (standardized)
text_sim,3.135953
lifestyle_score,1.671583
essay_len,0.145017
dist_penalty,0.000000
age_diff,-0.049669


,coef (standardized)
age_diff,-0.049669
dist_penalty,0.000000
essay_len,0.145017
lifestyle_score,1.671583
text_sim,3.135953


Start pool: 5000 (max_candidates=5000)
After orientation: 1561
Unfiltered: 1561 | After location filter (≤50 km): 1552
After location (≤50 km): 1552
After age rule: 1081
hinge_score available? True


In [5]:
# Comparisons - run OkCupid % and Hinge prob for the same user
# ---- knobs (match your OkCupid hard filters + strictness) ----
USER = 1234
TOPK = 20
MAX_KM = 50

HARD = {"diet": True, "drinks": True, "smokes": True, "drugs": True}
DRINKS_STRICT = False
SMOKES_STRICT = False
DRUGS_STRICT  = True

aligned = get_matches(user_id=USER, k=5000,use_orientation=True, use_location=True, max_km=MAX_KM,
    use_age=True, use_diet=HARD["diet"],
    use_drinks=HARD["drinks"],  drinks_strict=DRINKS_STRICT,
    use_smokes=HARD["smokes"],  smokes_strict=SMOKES_STRICT,
    use_drugs=HARD["drugs"],    drugs_strict=DRUGS_STRICT,
    allow_missing=True).copy()

# component scores used by both
aligned = compute_component_scores(user_id=USER, cands=aligned, max_km=MAX_KM,
    drinks_strict=DRINKS_STRICT, smokes_strict=SMOKES_STRICT, drugs_strict=DRUGS_STRICT,
    allow_missing=True).copy()

# ensure text_sim exists
if "text_sim" not in aligned.columns and "score" in aligned.columns:
    aligned["text_sim"] = aligned["score"]
aligned["text_sim"] = aligned["text_sim"].fillna(0)

# hinge-only features
if "age_diff" not in aligned.columns:
    q_age = float(df.loc[df["user_id"] == USER, "age"].iloc[0])
    aligned["age"] = aligned["age"].astype(float)
    aligned["age_diff"] = (aligned["age"] - q_age).abs()

if "essay_len" not in aligned.columns:
    bio = df.set_index("user_id").loc[aligned["user_id"], "bio_text"].fillna("").astype(str)
    aligned["essay_len"] = bio.str.len().to_numpy()

# dist_penalty for hinge (flat inside radius)
if "dist_penalty" not in aligned.columns and "distance_km" in aligned.columns:
    aligned["dist_penalty"] = np.maximum(0.0, aligned["distance_km"] - MAX_KM)

print("Aligned pool size:", len(aligned))

# ---- OkCupid scoring (fixed weights) on the same pool ----
okc_ranked = apply_weights(aligned.copy(), w_text=0.70, w_age=0.15, w_life=0.10, w_loc=0.05)
okc_ranked["compat_pct"] = (okc_ranked["final_score"] * 100).round(1)
okc = okc_ranked[["user_id","compat_pct","text_sim","age_score","lifestyle_score","loc_score",
    "distance_km","age","sex","orientation","location"]].head(TOPK).copy()

# ---- Hinge scoring (learned probability) on the same pool ----
X = aligned[hinge_model["X_cols"]].copy()
if "essay_len" in X.columns:
    X["essay_len"] = X["essay_len"].fillna(0).clip(lower=0)
if "dist_penalty" in X.columns:
    X["dist_penalty"] = X["dist_penalty"].fillna(0).clip(lower=0)

X_scaled = hinge_model["scaler"].transform(X)
aligned["prob_mutual_like"] = hinge_model["clf"].predict_proba(X_scaled)[:, 1]

def _fx(r):
    d = "?" if pd.isna(r.distance_km) else int(round(r.distance_km))
    pen = 0.0 if pd.isna(r.get("dist_penalty", 0.0)) else float(r.get("dist_penalty", 0.0))
    # flat-inside-radius location score for display only
    loc_flat = 1.0 if pen <= 0 else max(0.0, 1.0 - min(1.0, pen / MAX_KM))
    return (f"text={r.text_sim:.2f}, life={r.lifestyle_score:.2f}, "
            f"age={int(r.age_score)}, loc={loc_flat:.2f}, Δage={int(r.age_diff)}, dist~{d}km")

aligned["feature_breakdown"] = aligned.apply(_fx, axis=1)

hinge = aligned.sort_values("prob_mutual_like", ascending=False).head(TOPK)[["user_id","prob_mutual_like",
    "feature_breakdown","text_sim","lifestyle_score","age_score","dist_penalty","distance_km","age","sex",
    "orientation","location"]].copy()

print("OkCupid rows:", len(okc), "| Hinge rows:", len(hinge))
display(okc.head(3))
display(hinge.head(3))

Start pool: 5000 (max_candidates=5000)
After orientation: 2023
Unfiltered: 2023 | After location filter (≤50 km): 1717
After location (≤50 km): 1717
After age rule: 1342
Diet| before: 1342 → after: 1152
After diet: 1152
Drinks | before: 1152 → after: 1152 (strict=False)
After drinks (strict=False): 1152
Smokes | before: 1152 → after: 1117 (strict=False)
After smokes (strict=False): 1117
Drugs  | before: 1117 → after: 1008 (strict=True)
After drugs (strict=True): 1008
Aligned pool size: 1008
OkCupid rows: 20 | Hinge rows: 20


,user_id,compat_pct,text_sim,age_score,lifestyle_score,loc_score,distance_km,age,sex,orientation,location
13,57256,68.1,0.544072,1.0,1.0,1.0,41.230915,33.0,f,straight,"oakland, california"
16,53843,67.4,0.533671,1.0,1.0,1.0,41.230915,47.0,f,straight,"oakland, california"
18,4015,67.2,0.531963,1.0,1.0,1.0,43.696864,38.0,f,straight,"san francisco, california"


,user_id,prob_mutual_like,feature_breakdown,text_sim,lifestyle_score,age_score,dist_penalty,distance_km,age,sex,orientation,location
13,57256,0.324880,"text=0.54, life=1.00, age=1, loc=1.00, Δage=2,...",0.544072,1.0,1.0,0.0,41.230915,33.0,f,straight,"oakland, california"
18,4015,0.230392,"text=0.53, life=1.00, age=1, loc=1.00, Δage=7,...",0.531963,1.0,1.0,0.0,43.696864,38.0,f,straight,"san francisco, california"
16,53843,0.194443,"text=0.53, life=1.00, age=1, loc=1.00, Δage=16...",0.533671,1.0,1.0,0.0,41.230915,47.0,f,straight,"oakland, california"


In [ ]:
"""
Build one pool, score it two ways, show top-K

Knobs at the top
- USER, TOPK, MAX_KM = who we’re ranking, how many results to show, and the radius.
- HARD + strictness flags = which lifestyle filters are hard gates (must match) vs ignored.

Aligned pool
- get_matches(...) builds a single candidate pool for USER with identical hard gates for 
both methods: mutual orientation, age rule, within MAX_KM, and chosen lifestyle gates.
-compute_component_scores(...) adds the shared components both methods use: 
text_sim, age_score, lifestyle_score, loc_score and distance_km.

- Add Hinge-only features if missing:age_diff = |candidate age − user age| and essay_len = length of candidate’s bio
- Create dist_penalty = max(0, distance_km − MAX_KM) (flat score inside the radius).
This is what Hinge trains on now; OkCupid still uses loc_score for display.

OkCupid scoring (fixed weights)
- apply_weights(...) computes final_score from the components.
- Convert to percent → compat_pct.
- Slice top-TOPK and keep readable columns.

Hinge scoring (learned probability)
- Build X from exactly the model’s feature order hinge_model["X_cols"] 
(now: text_sim, lifestyle_score, age_diff, essay_len, dist_penalty).
- Clean only what Hinge used during training (essay_len, dist_penalty), 
then scale with the trained scaler and predict with the trained LR → prob_mutual_like.
- Add a readable feature_breakdown string. 
The displayed “loc” uses the flat-inside-radius rule (1.00 when inside; decays only if outside).

Output = two tables:
okc → top-K by compat_pct (OkCupid).
hinge → top-K by prob_mutual_like (Hinge).

In [6]:
# ---- Align + quick comparison metrics ----

# 1) Align by candidate user_id
left  = okc[["user_id", "compat_pct"]].rename(columns={"compat_pct": "okc_pct"})
right = hinge[["user_id", "prob_mutual_like"]].rename(columns={"prob_mutual_like": "hinge_prob"})
cmp = left.merge(right, on="user_id", how="outer")

# fill if a candidate shows up in only one list
cmp["okc_pct"]    = cmp["okc_pct"].fillna(0.0)
cmp["hinge_prob"] = cmp["hinge_prob"].fillna(0.0)

# 2) Ranks (1 = best) — compare on ranks so % vs prob are comparable
cmp["rank_okc"]   = cmp["okc_pct"].rank(method="min", ascending=False).astype(int)
cmp["rank_hinge"] = cmp["hinge_prob"].rank(method="min", ascending=False).astype(int)

# 3) Rank agreement & set overlap
spearman_like = cmp["rank_okc"].corr(cmp["rank_hinge"], method="pearson")  # Pearson on ranks ~ Spearman
set_okc, set_hinge = set(okc.user_id), set(hinge.user_id)
overlap = len(set_okc & set_hinge)
denom   = len(set_okc | set_hinge)
jaccard = (overlap / denom) if denom else 0.0

print(f"Spearman-like (rank agreement): {spearman_like:.3f}")
print(f"Overlap@{TOPK}: {overlap} / {TOPK}  |  Jaccard: {jaccard:.3f}")

# 4) Side-by-side view of the most mutually high-ranked
view = cmp.sort_values(["rank_okc", "rank_hinge"]).head(10)
display(view[["user_id", "okc_pct", "hinge_prob", "rank_okc", "rank_hinge"]])

# 5) Who appears in BOTH lists (ordered)
both = cmp[(cmp["rank_okc"] <= TOPK) & (cmp["rank_hinge"] <= TOPK)] \
         .sort_values(["rank_okc", "rank_hinge"])
print("In BOTH top-K (ordered):")
display(both[["user_id", "okc_pct", "hinge_prob", "rank_okc", "rank_hinge"]])

# 6) Quick diagnostics — why disagreements happen
try:
    okc_reasons   = okc[["user_id", "text_sim", "lifestyle_score", "loc_score", "distance_km"]]
    hinge_reasons = hinge[["user_id", "feature_breakdown", "dist_penalty"]]
    diag = (cmp.merge(okc_reasons, on="user_id", how="left")
              .merge(hinge_reasons, on="user_id", how="left"))
    display(diag.sort_values(["rank_okc", "rank_hinge"]).head(10))
except Exception as e:
    print("Diag join skipped:", e)


Spearman-like (rank agreement): 0.971
Overlap@20: 19 / 20  |  Jaccard: 0.905


,user_id,okc_pct,hinge_prob,rank_okc,rank_hinge
20,57256,68.1,0.324880,1,1
18,53843,67.4,0.194443,2,3
3,4015,67.2,0.230392,3,2
9,30039,67.2,0.186397,3,4
6,9558,66.6,0.110717,5,7
7,20404,66.3,0.102166,6,8
16,49650,66.2,0.153976,7,5
10,33605,66.1,0.084588,8,11
5,7668,66.0,0.112203,9,6
14,42403,65.9,0.094043,10,9


In BOTH top-K (ordered):


,user_id,okc_pct,hinge_prob,rank_okc,rank_hinge
20,57256,68.1,0.324880,1,1
18,53843,67.4,0.194443,2,3
3,4015,67.2,0.230392,3,2
9,30039,67.2,0.186397,3,4
6,9558,66.6,0.110717,5,7
7,20404,66.3,0.102166,6,8
16,49650,66.2,0.153976,7,5
10,33605,66.1,0.084588,8,11
5,7668,66.0,0.112203,9,6
14,42403,65.9,0.094043,10,9


,user_id,okc_pct,hinge_prob,rank_okc,rank_hinge,text_sim,lifestyle_score,loc_score,distance_km,feature_breakdown,dist_penalty
20,57256,68.1,0.324880,1,1,0.544072,1.0,1.0,41.230915,"text=0.54, life=1.00, age=1, loc=1.00, Δage=2,...",0.0
18,53843,67.4,0.194443,2,3,0.533671,1.0,1.0,41.230915,"text=0.53, life=1.00, age=1, loc=1.00, Δage=16...",0.0
3,4015,67.2,0.230392,3,2,0.531963,1.0,1.0,43.696864,"text=0.53, life=1.00, age=1, loc=1.00, Δage=7,...",0.0
9,30039,67.2,0.186397,3,4,0.531841,1.0,1.0,43.696864,"text=0.53, life=1.00, age=1, loc=1.00, Δage=7,...",0.0
6,9558,66.6,0.110717,5,7,0.522563,1.0,1.0,48.458096,"text=0.52, life=1.00, age=1, loc=1.00, Δage=15...",0.0
7,20404,66.3,0.102166,6,8,0.518943,1.0,1.0,43.696864,"text=0.52, life=1.00, age=1, loc=1.00, Δage=12...",0.0
16,49650,66.2,0.153976,7,5,0.517111,1.0,1.0,41.230915,"text=0.52, life=1.00, age=1, loc=1.00, Δage=0,...",0.0
10,33605,66.1,0.084588,8,11,0.515604,1.0,1.0,30.936387,"text=0.52, life=1.00, age=1, loc=1.00, Δage=3,...",0.0
5,7668,66.0,0.112203,9,6,0.513741,1.0,1.0,41.230915,"text=0.51, life=1.00, age=1, loc=1.00, Δage=12...",0.0
14,42403,65.9,0.094043,10,9,0.513060,1.0,1.0,41.230915,"text=0.51, life=1.00, age=1, loc=1.00, Δage=1,...",0.0


In [8]:
"""
Spearman-like = 0.971 → the two rankings are almost identical in order.
Overlap@20 = 19/20 and Jaccard = 0.905 → only 1 candidate differs between the lists; 
the sets are >90% the same.

Why they match so closely
- We forced the same hard pool (orientation, age rule, ≤50 km).
- Hinge is trained with labels that encode the same rule-of-thumb as OkCupid 
(text-heavy, lifestyle important, age must pass, flat inside radius).
-Learned weights ended up similar to your fixed weights (text ≫ lifestyle ≫ small nudges), 
so the probability is almost a monotonic transform of the OkCupid score.
"""

'\nSpearman-like = 0.971 → the two rankings are almost identical in order.\nOverlap@20 = 19/20 and Jaccard = 0.905 → only 1 candidate differs between the lists; \nthe sets are >90% the same.\n\nWhy they match so closely\n- We forced the same hard pool (orientation, age rule, ≤50 km).\n- Hinge is trained with labels that encode the same rule-of-thumb as OkCupid \n(text-heavy, lifestyle important, age must pass, flat inside radius).\n-Learned weights ended up similar to your fixed weights (text ≫ lifestyle ≫ small nudges), \nso the probability is almost a monotonic transform of the OkCupid score.\n'

In [10]:
"""
Align the two lists and compute comparison metrics

Align on user_id
- Merge okc and hinge on user_id → cmp.

Rank on each metric
- Convert scores to ranks (1 = best) so percent vs probability are comparable.

Agreement & overlap
- spearman_like = Pearson correlation of the two rank columns (≈ Spearman); closer to 1 means same ordering.
- overlap = size of the intersection of the two top-K sets.
- jaccard = overlap / union (0–1), measures set similarity.

Diagnostics
-Join back quick reasons:
    From OkCupid: text_sim, lifestyle_score, loc_score, distance_km
    From Hinge: feature_breakdown and dist_penalty
"""

'\nAlign the two lists and compute comparison metrics\n\nAlign on user_id\n- Merge okc and hinge on user_id → cmp.\n\nRank on each metric\n- Convert scores to ranks (1 = best) so percent vs probability are comparable.\n\nAgreement & overlap\n- spearman_like = Pearson correlation of the two rank columns (≈ Spearman); closer to 1 means same ordering.\n- overlap = size of the intersection of the two top-K sets.\n- jaccard = overlap / union (0–1), measures set similarity.\n\nDiagnostics\n-Join back quick reasons:\n    From OkCupid: text_sim, lifestyle_score, loc_score, distance_km\n    From Hinge: feature_breakdown and dist_penalty\n'